# Creation NER Dataset

Creation of the dataset in **IOB format** for **NER tasks**.
The dataset will be used to fine-tune the **BIOClinicalBert** model, or a model from the BERT family.

In this case, there will be only one entity: **TARGET**.

## Imports and environment setup

In [ ]:
!pip install together -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.3/91.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.5 MB/s eta 0:00:00


In [ ]:
import os
import re
import json
import numpy as np
import pandas as pd
import asyncio
import together
from together import AsyncTogether, Together
import time
import string
import plotly.express as px
from collections import Counter
import pandas as pd
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
from together import Together
async_client = AsyncTogether()

## Constants

In [ ]:
DATA_PATH='/content/drive/MyDrive/SanRaffaele/Data/Dataset/2019/lettere_dimissioni_2019'
OUTPUT_PATH='/content/drive/MyDrive/SanRaffaele/Data/Dataset NER'

In [ ]:
# Stopwords e punteggiatura da ignorare o marcare come 'O'
STOPWORDS = {'a', 'the', 'an', 'in', 'on', 'and', 'of', 'for', 'with', 'at', 'by', 'to', 'from', '-', 'was'}

In [ ]:
promp_base='''Sei un medico e voglio che assegni ad ogni parola una label seguendo il formato IOB (Inside-Outside-Beginning),
usato per i task di Named Entity Recognition (NER), alla seguente frase presa da una lettera di dimissioni.

L' **unica label** da assegnare è:
**TARGET**

dove le entità target sono le seguenti:

### Mappa delle entità e tipi

| Nome                                      | Descrizione                                                                                          |
|-------------------------------------------|------------------------------------------------------------------------------------------------------|
| n_cartella                                | Numero identificativo univoco assegnato alla cartella clinica del paziente.                         |
| data_ingresso_cch                         | Data in cui il paziente è stato ricoverato presso il reparto di Cardiochirurgia.                   |
| data_dimissione_cch                       | Data in cui il paziente è stato dimesso dal reparto di Cardiochirurgia.                            |
| nome                                      | Nome proprio del paziente.                                                                          |
| cognome                                   | Cognome del paziente.                                                                               |
| sesso                                     | Sesso biologico del paziente (M = Maschio, F = Femmina).                                            |
| numero di telefono                        | Recapito telefonico del paziente o di un contatto di riferimento.                                  |
| età al momento dell'intervento            | Età del paziente calcolata alla data dell’intervento chirurgico.                                   |
| data_di_nascita                           | Data di nascita del paziente.                                                                       |
| Diagnosi                                  | Diagnosi principale alla base dell'indicazione chirurgica.                                          |
| Anamnesi                                  | Anamnesi patologica remota e prossima, utile per la valutazione del rischio operatorio.            |
| Motivo ricovero                           | Indicazione clinica per il ricovero in Cardiochirurgia.                                             |
| classe_nyha                               | Classe funzionale NYHA per scompenso cardiaco (I-IV), definisce la gravità dei sintomi.            |
| angor                                     | Presenza di angina pectoris (dolore toracico di origine ischemica).                                |
| STEMI/NSTEMI                              | Presenza di infarto miocardico acuto con/senza sopraslivellamento del tratto ST.                   |
| scompenso_cardiaco_nei_3_mesi_precedenti  | Episodi di scompenso cardiaco documentati nei 3 mesi precedenti l’intervento.                      |
| fumo                                      | Abitudine al fumo (0 = mai fumato, 1 = ex-fumatore, 2 = fumatore attivo).                          |
| diabete                                   | Presenza di diabete mellito noto.                                                                  |
| ipertensione                              | Presenza di ipertensione arteriosa.                                                                |
| dislipidemia                              | Presenza di dislipidemia (colesterolo e/o trigliceridi elevati).                                   |
| BPCO                                      | Presenza di broncopneumopatia cronica ostruttiva.                                                  |
| stroke_pregresso                          | Precedente episodio di ictus cerebrale ischemico o emorragico.                                     |
| TIA_pregresso                             | Episodio pregresso di attacco ischemico transitorio (TIA).                                         |
| vasculopatiaperif                         | Malattia vascolare periferica documentata (es. arteriopatia arti inferiori).                       |
| neoplasia_pregressa                       | Presenza di neoplasie trattate in passato.                                                         |
| irradiazionetoracica                      | Pregressa radioterapia al torace, rilevante per effetti tardivi su cuore e vasi.                   |
| insufficienza_renale_cronica              | Presenza di insufficienza renale cronica diagnosticata.                                            |
| familiarita_cardiovascolare               | Familiarità per malattie cardiovascolari premature.                                                |
| limitazione_mobilita                      | Presenza di limitazioni significative alla mobilità (es. pazienti allettati).                      |
| endocardite                               | Pregressa o attiva endocardite infettiva.                                                          |
| ritmo_all_ingresso                        | Ritmo cardiaco al momento del ricovero (0 = ritmo sinusale, 1 = FA, 2 = altro).                    |
| fibrillazione_atriale                     | Presenza di fibrillazione atriale (0 = mai, 1 = parossistica, 2 = permanente/persistente).         |
| dialisi                                   | Paziente in trattamento emodialitico o peritoneale.                                                |
| elettivo_urgenza_emergenza                | Tipo di intervento (0 = elettivo, 1 = urgente, 2 = emergenza).                                     |
| pm                                        | Presenza di pacemaker.                                                                             |
| crt                                       | Presenza di terapia di resincronizzazione cardiaca (CRT).                                          |
| icd                                       | Presenza di defibrillatore impiantabile (ICD).                                                     |
| pci_pregressa                             | Precedente angioplastica coronarica percutanea (PCI).                                              |
| REDO                                      | Intervento cardiochirurgico di revisione (non prima chirurgia).                                   |
| Anno REDO                                 | Anno in cui è stato eseguito l'intervento REDO precedente.                                         |
| Tipo di REDO                              | Descrizione del tipo di intervento REDO eseguito.                                                  |
| Terapia                                   | Terapia farmacologica in atto al momento del ricovero.                                             |
| lasix                                     | Uso documentato di furosemide (Lasix).                                                             |
| lasix_dosaggio                            | Dosaggio giornaliero di furosemide in mg.                                                          |
| nitrati                                   | Assunzione di nitrati (vasodilatatori usati per l'angina).                                         |
| antiaggregante                            | Presenza di terapia antiaggregante (es. ASA, clopidogrel).                                         |
| dapt                                      | Doppia antiaggregazione piastrinica (es. ASA + clopidogrel/prasugrel).                             |
| anticoagorali                             | Terapia anticoagulante in corso (es. warfarin, DOAC).                                              |
| aceinib                                   | Uso di ACE-inibitori.                                                                             |
| betabloc                                  | Uso di beta-bloccanti.                                                                            |
| sartanici                                 | Uso di sartani (ARBs).                                                                            |
| caantag                                   | Uso di calcio-antagonisti.                                                                        |
| esami_all_ingresso                        | Risultati di laboratorio e strumentali al momento dell’ingresso.                                   |
| Decorso_post_operatorio                   | Descrizione del decorso clinico successivo all’intervento chirurgico.                              |
| IABP/ECMO/IMPELLA                         | Necessità di supporto meccanico circolatorio (IABP, ECMO o Impella).                              |
| Inotropi                                  | Necessità di farmaci inotropi positivi nel post-operatorio.                                       |
| secondo_intervento                        | Esecuzione di un secondo intervento durante la degenza attuale.                                   |
| Tipo_secondo_intervento                   | Tipo e motivazione del secondo intervento chirurgico.                                              |
| II_Run                                    | Presenza di secondo passaggio in circolazione extracorporea (CEC).                                |
| Causa_II_Run_CEC                          | Motivazione per il secondo utilizzo della CEC.                                                    |
| LCOS                                      | Sindrome da bassa portata cardiaca (Low Cardiac Output Syndrome) post-operatoria.                  |
| Impianto_PM_post_intervento               | Necessità di impianto di pacemaker dopo l’intervento.                                             |
| Stroke_TIA_post_op                        | Evento neurologico ischemico (TIA/stroke) avvenuto dopo l’intervento.                             |
| Necessità_di_trasfusioni                  | Necessità di trasfusioni ematiche post-intervento.                                                |
| IRA                                       | Insufficienza renale acuta insorta nel post-operatorio.                                           |
| Insufficienza_respiratoria                | Insorgenza di insufficienza respiratoria nel post-operatorio.                                     |
| FA_di_nuova_insorgenza                    | Fibrillazione atriale di nuova insorgenza nel post-operatorio.                                    |
| Ritmo_alla_dimissione                     | Ritmo cardiaco documentato alla dimissione (0 = sinusale, 1 = FA, 2 = altro).                      |
| H_Stay_giorni (da intervento a dimissione)| Durata della degenza in giorni, calcolata dall’intervento alla dimissione.                        |
| Morte                                     | Evento di decesso durante la degenza cardiochirurgica.                                            |
| Causa_morte                               | Causa clinica del decesso (es. sepsi, shock cardiogeno, ecc.).                                    |
| data_morte                                | Data del decesso, se avvenuto.                                                                    |
| esami_alla_dimissione                     | Risultati di laboratorio e strumentali prima della dimissione.                                    |
| terapia_alla_dimissione                   | Terapia farmacologica prescritta alla dimissione.                                                 |


**ATTENZIONE**:
- Quando stai assegnando la label considera sia sia il nome dell'entità che il suo il valore al'interno della stessa entità TARGET
- Non estrarre **nessuna entità TARGET** diversa da quelle elencate.
- Attenzione però i nomi delle entità target che vedi sopra sono in alcuni casi degli acronimi o diminutivi delle entità
- Il numero di parole nella frase deve essere **esattamente uguale** al numero di label corrispondenti.
- Il risultato deve essere in formato JSON, come una lista di oggetti, ciascuno con le seguenti due chiavi:
  - `"frase"`: stringa della frase processata.
  - `"label"`: lista di label IOB corrispondenti alle parole.
IL NUMERO DI PAROLE NELLA FRASE DEVE ESSERE ESATTAMENTE UGUALE AL NUMERO DI LABEL NELL'ALTRA COLONNA


**NON AGGIUNGERE COMMENTI, NOTE O SPIEGAZIONI**, solo la lista JSON.

---

###Esempi di input

Si dimette in data 02/09/2019
il Sig. BERTOLOTTI FRANCO
Nato il 27/03/1939 telefono 3479927663
ricoverato presso questo ospedale dal 27/08/2019
Numero Cartella 2019034139
Intervento di plastica valvolare mitralica per via percutanea mediante posizionamento di duplice dispositivo Mitraclip.
Insufficienza mitralica in status post rivascolarizzazione miocardica chirurgica mediante triplice bypass coronarico.
Paziente nega farmacoallergie.
Familiarità positiva per cardiopatia ischemica (padre).
Ex fumatore, stop nel 1990 (1 pack/die).
Diabete mellito in tp ipoglicemizzante orale.
IRC (crea all'ingresso 2,64 mg/dl).


---

###Esempi output(esempio parziale in JSON):

```json

  {
    "frase": "Si dimette in data 02/09/2019",
    "label": ["O", "O", "O", "O", "B-TARGET"]
  },
  {
    "frase": "il Sig. BERTOLOTTI FRANCO",
    "label": ["O", "O", "B-TARGET", "B-TARGET"]
  },
  {
    "frase": "Nato il 27/03/1939 telefono 3479927663",
    "label": ["O", "O", "B-TARGET", "O", "B-TARGET"]
  },
  {
    "frase": "ricoverato presso questo ospedale dal 27/08/2019",
    "label": ["O", "O", "O", "O", "O", "B-TARGET"]
  },
  {
    "frase": "Numero Cartella 2019034139",
    "label": ["O", "O", "B-TARGET"]
  },
  {
    "frase": "Intervento di plastica valvolare mitralica per via percutanea mediante posizionamento di duplice dispositivo Mitraclip.",
    "label": ["B-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET"]
  },
  {
    "frase": "Insufficienza mitralica in status post rivascolarizzazione miocardica chirurgica mediante triplice bypass coronarico.",
    "label": ["B-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET"]
  },
  {
    "frase": "Paziente nega farmacoallergie.",
    "label": ["B-TARGET", "I-TARGET", "I-TARGET"]
  },
  {
    "frase": "Familiarità positiva per cardiopatia ischemica (padre).",
    "label": ["B-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET"]
  },
  {
    "frase": "Ex fumatore, stop nel 1990 (1 pack/die).",
    "label": ["B-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET"]
  },
  {
    "frase": "Diabete mellito in tp ipoglicemizzante orale.",
    "label": ["B-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET"]
  },
  {
    "frase": "IRC (crea all'ingresso 2,64 mg/dl).",
    "label": ["B-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET", "I-TARGET"]
  }

  frase da processare:
'''

## Functions

In [ ]:
def extract_text_md(folder_path):
  list_file=os.listdir(folder_path)
  text_list=[]
  for file in list_file:
    path_file=os.path.join(folder_path,file)
    with open(path_file,'r') as f:
      text_list.append(f.read())
  return text_list

In [ ]:
def clean_text(text: str) -> str:
    # Rimuove immagini ![](...)
    text = re.sub(r'!\[\]\([^\)]+\)', '', text)

    # Rimuove separatori tipo {5}---- o ### Titoli
    text = re.sub(r'\{[0-9]+\}[-=]+', '', text)
    text = re.sub(r'#{1,6}', '', text)

    # Rimuove righe vuote e spazi multipli
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    # Rimuove tabelle markdown
    text = re.sub(r'\|.*\|', '', text)

    # Rimuove intestazioni ripetute tipo "stampato il...", "Pag", etc.
    text = re.sub(r'IRCCS.*?Pag [0-9]+ / [0-9]+.*', '', text)

    return text.strip()

In [ ]:
def split_into_sentences(text: str) -> list:
    # Prima normalizzazione
    text = clean_text(text)

    # Separazione in frasi
    sentences = text.split('\n')

    # Pulizia finale di ogni frase
    sentences = [s.strip() for s in sentences if s.strip()]

    return sentences

In [ ]:
async def get_response(prompt_text:str, model:str)->str:
  for sleep_time in [1, 2, 4]:
    try:
      response = await async_client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt_text}],
            response_format={"type": "json_schema"},
            temperature=0.1,
            top_p=1,
            max_tokens=8192
            )
      break
    except together.error.RateLimitError as e:
        print(e)
        await asyncio.sleep(sleep_time)

  response_text=response.choices[0].message.content
  return response_text

In [ ]:
def create_dataframe(response_list):
    data = []

    for item in response_list:
        try:
            parsed = json.loads(item)

            if isinstance(parsed, list):
                if len(parsed) > 1:
                    lunghezze=[len(frase.split()) for frase in parsed]# se ci sono delle risposte parola: label,...,parola:label
                    conteggio=Counter(lunghezze)
                    if conteggio['1']>3:
                      for k, v in parsed.items():
                        frase+=k
                        label+=v
                      new_item = {
                          "frase": frase,
                          "label": label
                      }
                      data.append(new_item)

                    else:
                      data.extend(parsed)
                elif len(parsed) == 1:
                    data.append(parsed[0])
                else:
                    print("Lista vuota:", parsed)
            else:
                if len(parsed.split('{'))>2:#vuol dire che c'è più di un dizionario ma che mancano le []:
                  parsed=[parsed]
                  data.extend(parsed)
                else:
                  data.append(parsed)

        except json.JSONDecodeError as e:
            print("Errore di parsing JSON:", e)
            continue
        except Exception as e:
            print("Errore generico:", e)
            continue

    return pd.DataFrame(data)


In [ ]:
def save_csv(df:pd.DataFrame, path:str):
  df.to_csv(path, index=False)

In [ ]:
def plot_distribuzione_lunghezza_frasi(dataset):
    # Calcola la lunghezza (in parole) di ciascuna frase
    lunghezze = [len(frase.split()) for frase in dataset]

    # Conta quante frasi hanno ciascuna lunghezza
    conteggio = Counter(lunghezze)

    # Ordina i dati per numero di parole
    x = sorted(conteggio.keys())
    y = [conteggio[k] for k in x]

    # Crea il grafico
    fig = px.bar(
        x=x,
        y=y,
        labels={'x': 'Numero di parole', 'y': 'Numero di frasi'},
        title='Distribuzione della lunghezza delle frasi (in parole)'
    )
    fig.show()

In [ ]:
def count_mismatch(df):
    new_data = []
    count=0
    for index, row in df.iterrows():
        sentence = row['frase']
        labels = row['label']

        words = sentence.split()

        if len(words) != len(labels):
            print(f"Mismatch in row {index}, sentence '{sentence}' {labels}. {len(words)}-{len(labels)}")
            count=count+1
            continue

    print(f'\nNumber of mismatch: {count}')

In [ ]:
def is_stopword_or_punct(token):
    return (
        token.lower() in STOPWORDS or
        all(c in string.punctuation for c in token)
    )

# Funzione per riallineare
def align_tokens_and_labels(row):
    tokens = row['frase'].split()
    labels = row['label']

    if len(tokens) == len(labels):
        return labels  # già ok

    new_labels = []
    label_idx = 0
    for token in tokens:
        if label_idx < len(labels):
           #caso isolato particolare sta una label così boh perchè
            if labels[label_idx]=='!-TARGET':
              labels[label_idx]='I-TARGET'
            # se token è significativo, usa label
            if not is_stopword_or_punct(token):
                new_labels.append(labels[label_idx])
                label_idx += 1
            else:
                # se token è stopword/punteggiatura, metti 'O'
                new_labels.append('O')
        else:
            # se hai finito le label, riempi con 'O'
            new_labels.append('O')

    return new_labels

In [ ]:
def create_augment_dataframe(df):
  new_data = []
  for index, row in df.iterrows():
      sentence = row['frase']
      labels = row['label']

      words = sentence.split()

      for i in range(len(words)):
          new_data.append([index, words[i], labels[i], sentence])
  return pd.DataFrame(new_data, columns=["#Sentence", "Word", "Label", "Sentence"])


In [ ]:
def get_tags(frequencies):
  tags = {}
  for tag, count in zip(frequencies.index, frequencies):
      if tag != "O":
          if tag[2:] not in tags.keys():
              tags[tag[2:]] = count
          else:
              tags[tag[2:]] += count
      continue
  print(tags)
  return tags

In [ ]:
def clean_df(df):
  rows_to_drop = []
  for index, row in df.iterrows():
    if isinstance(row['frase'], float):
        rows_to_drop.append(index)

    elif len(row['label']) <1 or len(row['frase']) < 15:
        rows_to_drop.append(index)

  # Drop rows after the loop
  df.drop(index=rows_to_drop, inplace=True)


In [ ]:
def find_sentece2delete(new_df):
  # Calcola numero di target (etichette diverse da 'O')
  target_count = (new_df['Label'] == 'B-TARGET').sum()

  # Raggruppa le frasi per ID e trova quelle con solo 'O'
  only_o_sentences = []
  grouped = new_df.groupby('#Sentence')

  for sentence_id, group in grouped:
      labels = group['Label'].tolist()
      if all(label == 'O' for label in labels):
          only_o_sentences.append((sentence_id, len(labels)))  # (id, count of 'O')

  # Ora scegli randomicamente alcune finché il numero totale di 'O' == target_count
  random.shuffle(only_o_sentences)

  selected_ids = []
  o_count = 0

  for sent_id, o_len in only_o_sentences:
      if o_count + o_len > target_count:
          continue  # salta se sforerebbe
      selected_ids.append(sent_id)
      o_count += o_len
      if o_count == target_count:
          break

  # Estrai le righe dal df con quei sentence ID
  #new_df = new_df[new_df['#Sentence'].isin(selected_ids)]

  print(f"Numero target = {target_count}, Numero di 'O' selezionati = {o_count}")
  print(f"Frasi selezionate: {len(selected_ids)}")

  return selected_ids


In [ ]:
def remove_sentences(selected_ids,df):
  selected_sentences = (
    new_df[new_df['#Sentence'].isin(selected_ids)]
    .groupby('#Sentence')['Word']
    .apply(lambda x: ' '.join(x))
    .tolist()
    )
  for frase in selected_sentences:
      idx = df[df['frase'] == frase].index
      if len(idx) > 0:
          df.drop(idx[0], inplace=True)


## Dataset Creation

We have 291 discharge letter

In [ ]:
file_list=extract_text_md(DATA_PATH)
print(len(file_list))

291


We divide the text of each discharge letter into sentence, because in this way for the LLM will be more easy to assign the labels TARGET, O

In [ ]:
for file in file_list:
  frasi=split_into_sentences(file)
  file_list[file_list.index(file)]=frasi

In [ ]:
file_list=[file for file in file_list if len(file)>0]
print(len(file_list))

281


We append all the phrases to a list and plot the distribution of their lengths to get a better overview of our dataset.

In [ ]:
dataset = [frase for file in file_list for frase in file]
len(dataset)

21051

In [ ]:
plot_distribuzione_lunghezza_frasi(dataset)

### LLM inference

In [ ]:
response_list=[]
fine =0
inizio=0

In [ ]:
prompt_list=[promp_base+frase for frase in dataset]

while fine<len(prompt_list):
  fine=inizio+20
  print(fine)
  if fine>len(prompt_list):
    fine=len(prompt_list)
  for i in range(10):
    try:
      response_list.extend(await asyncio.gather(*[get_response(prompt, 'meta-llama/Llama-3.3-70B-Instruct-Turbo') for prompt in prompt_list[inizio:fine]]))
      break
    except Exception as e:
      print(f"Tentativo {i+1} fallito: {e}")
      time.sleep(2 ** i)
  time.sleep(1)
  inizio=fine
  np.save(OUTPUT_PATH+'/response_list.npy', response_list)



20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
Tentativo 1 fallito: Error code: 503 - The server is overloaded or not ready yet.
Tentativo 2 fallito: Error code: 503 - The server is overloaded or not ready yet.
Tentativo 3 fallito: Error code: 503 - The server is overloaded or not ready yet.
Tentativo 4 fallito: Error code: 503 - The server is overloaded or not ready yet.
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
Tentativo 1 fallito: Error code: 503 - The server is overloa

## Analysis of the output

We load the dataset created with the LLM

In [ ]:
path=os.listdir(OUTPUT_PATH)[0]
response_list=np.load(os.path.join(OUTPUT_PATH,path))

In [ ]:
response_list.size

21051

We create a dataframe from the responses to manage the data more easily.

In [ ]:
df=create_dataframe(response_list)

Streaming output truncated to the last 5000 lines.
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore di parsing JSON: Unterminated string starting at: line 40 column 16155 (char 19966)
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore generico: 'dict' object has no attribute 'split'
Errore generico: '

In [ ]:
df.head()

,frase,label
0,Dipartimento Cardiotoracovascolare U.O. Cardio...,"[O, O, O, O, O, O, O, B-TARGET]"
1,RELAZIONE CLINICA alla DIMISSIONE - Definitiva,"[O, O, O, O, O, O]"
2,Alla cortese attenzione del medico curante,"[O, O, O, O, O, O]"
3,Si dimette in data 02/09/2019,"[O, O, O, O, B-TARGET]"
4,il Sig. BERTOLOTTI FRANCO,"[O, O, B-TARGET, I-TARGET]"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15823 entries, 0 to 15822
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   frase   15823 non-null  object
 1   label   15823 non-null  object
dtypes: object(2)
memory usage: 247.4+ KB


### Clean output

One of the main problems we can face when creating a dataset with this approach is that the number of labels may differ from the number of words. This happens because the LLM does not perform true classification; instead, as an autoregressive model, it simply generates the next word.

For this reason we count the mismatch and then we fix them.

In [ ]:
count_mismatch(df)

Streaming output truncated to the last 5000 lines.
Mismatch in row 5800, sentence 'Prof. A. Castiglioni Prof. M. De Bonis Prof. O. Alfieri Dr.ssa M. Calabrese' ['B-TARGET', 'I-TARGET', 'I-TARGET', 'B-TARGET', 'I-TARGET', 'I-TARGET', 'B-TARGET', 'I-TARGET', 'I-TARGET', 'B-TARGET', 'I-TARGET', 'I-TARGET']. 13-12
Mismatch in row 5801, sentence 'Timbro'e firma del medico dimettente (la firma digitale in calce, se presente, sostituisce la firma autografa)' ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']. 16-21
Mismatch in row 5803, sentence 'Unità Cardiochirurgia delle Terapie Avanzate e di Ricerca Direttore Prof. Michele De Bonis' ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-TARGET', 'I-TARGET']. 13-10
Mismatch in row 5810, sentence 'Diagnosi alla dimissione' ['B-TARGET', 'I-TARGET']. 3-2
Mismatch in row 5814, sentence 'Cenni Anamnestici' ['B-TARGET']. 2-1
Mismatch in row 5816, sentence 'Ultimo ecocardiogramma TT (2/19): FE 55%, SIM 

In [ ]:
df['label'] = df.apply(align_tokens_and_labels, axis=1)

In [ ]:
count_mismatch(df)


Number of mismatch: 0


In [ ]:
df.head()

,frase,label
0,Dipartimento Cardiotoracovascolare U.O. Cardio...,"[O, O, O, O, O, O, O, B-TARGET]"
1,RELAZIONE CLINICA alla DIMISSIONE - Definitiva,"[O, O, O, O, O, O]"
2,Alla cortese attenzione del medico curante,"[O, O, O, O, O, O]"
3,Si dimette in data 02/09/2019,"[O, O, O, O, B-TARGET]"
4,il Sig. BERTOLOTTI FRANCO,"[O, O, B-TARGET, I-TARGET]"


In [ ]:
new_df=create_augment_dataframe(df)
new_df

,#Sentence,Word,Label,Sentence
0,0,Dipartimento,O,Dipartimento Cardiotoracovascolare U.O. Cardio...
1,0,Cardiotoracovascolare,O,Dipartimento Cardiotoracovascolare U.O. Cardio...
2,0,U.O.,O,Dipartimento Cardiotoracovascolare U.O. Cardio...
3,0,Cardiochirurgia,O,Dipartimento Cardiotoracovascolare U.O. Cardio...
4,0,Direttore,O,Dipartimento Cardiotoracovascolare U.O. Cardio...
...,...,...,...,...
127467,15822,"presente,",O,Timbro e firma del medico dimettente (la firma...
127468,15822,sostituisce,O,Timbro e firma del medico dimettente (la firma...
127469,15822,la,O,Timbro e firma del medico dimettente (la firma...
127470,15822,firma,O,Timbro e firma del medico dimettente (la firma...


In [ ]:
print(f"Number of tags:{len(new_df['Label'].unique())}")
frequencies = new_df['Label'].value_counts()
print(frequencies)

Number of tags:3
Label
O           70848
I-TARGET    42653
B-TARGET    13971
Name: count, dtype: int64


In [ ]:
tags=get_tags(frequencies)

{'TARGET': 56624}


We remove the phrases hat have less than 15 char or they have just one entity

In [ ]:
clean_df(df)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15254 entries, 0 to 15822
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   frase   15254 non-null  object
 1   label   15254 non-null  object
dtypes: object(2)
memory usage: 357.5+ KB


We select  and remove the phrases that have jus O label in order to balance the labes

In [ ]:
remove_sentences(find_sentece2delete(new_df),df)

Numero target = 13971, Numero di 'O' selezionati = 13971
Frasi selezionate: 1836


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13544 entries, 3 to 15822
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   frase   13544 non-null  object
 1   label   13544 non-null  object
dtypes: object(2)
memory usage: 317.4+ KB


In [ ]:
new_df=create_augment_dataframe(df)


In [ ]:
print(f"Number of tags:{len(new_df['Label'].unique())}")
frequencies = new_df['Label'].value_counts()
print(frequencies)

Number of tags:3
Label
O           56712
I-TARGET    42405
B-TARGET    13728
Name: count, dtype: int64


## Save Dataset

In [ ]:
save_csv(df,OUTPUT_PATH+'/clean_NER_LLAMA70B.csv')